In [4]:
import torch
from PIL import Image
from transformers import AutoImageProcessor, AutoModel, AutoTokenizer, AutoProcessor, AutoModelForSeq2SeqLM
import faiss
import numpy as np
import json
from datasets import load_dataset
import os

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained("openai/clip-vit-base-patch16").to(device)
processor = AutoImageProcessor.from_pretrained("openai/clip-vit-base-patch16")
tokenizer = AutoTokenizer.from_pretrained("openai/clip-vit-base-patch16")

/opt/miniconda3/envs/vilbert-mt/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [4]:
import os
print(os.getcwd())

/Users/filiprejmus/repos/multimodalqa/baselines/llm_qa


In [5]:
import json
from datasets import Dataset, Features, Image, Value
from PIL import Image as PILImage
import os

def load_and_validate_image(image_path):
    try:
        with PILImage.open(image_path) as img:
            img.verify()  # Verify that it's a valid image
        return image_path
    except (IOError, SyntaxError):
        return None  # Return None for corrupt or missing images

def load_dataset(metadata_file, image_folder):
    data = []
    
    with open(metadata_file, 'r') as f:
        for line in f:
            item = json.loads(line)
            image_path = os.path.join(image_folder, item['path'])
            
            # Validate and load the image
            validated_image_path = load_and_validate_image(image_path)
            
            if validated_image_path:
                item['image'] = validated_image_path
                data.append(item)
            else:
                print(f"Warning: Image not found or corrupt: {image_path}")
    
    return data

# Set your file paths
metadata_file = '../downloads/MMQA/MMQA_images.jsonl'
image_folder = '../downloads/MMQA/final_dataset_images'

# Load and process the data
dataset_dict = load_dataset(metadata_file, image_folder)

# Define the features of your dataset
features = Features({
    'title': Value('string'),
    'url': Value('string'),
    'id': Value('string'),
    'path': Value('string'),
    'image': Image()
})

# Create the dataset
dataset = Dataset.from_dict({k: [d[k] for d in dataset_dict] for k in dataset_dict[0].keys()}, features=features)

# Print some information about the dataset
print(dataset)
print(f"Number of samples: {len(dataset)}")

# You can now use this dataset or save it
# dataset.save_to_disk("my_image_dataset")
# Or push to the Hugging Face Hub
# dataset.push_to_hub("username/dataset-name")

/opt/miniconda3/envs/vilbert-mt/lib/python3.11/site-packages/PIL/Image.py:3218: DecompressionBombWarning: Image size (129967200 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Dataset({
    features: ['title', 'url', 'id', 'path', 'image'],
    num_rows: 57052
})
Number of samples: 57052


In [2]:
# Load model directly
from transformers import BlipProcessor, BlipForConditionalGeneration

processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large")

/opt/miniconda3/envs/vilbert-mt/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/miniconda3/envs/vilbert-mt/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
inputs = processor(dataset[0]['image'], return_tensors="pt")

In [8]:
out = model.generate(**inputs)

/opt/miniconda3/envs/vilbert-mt/lib/python3.11/site-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [9]:
print(processor.decode(out[0], skip_special_tokens=True))

a red and white elephant with stars in the middle of it


In [10]:
dataset[0]

{'title': 'Republican Party (United States)',
 'url': 'https://en.wikipedia.org/wiki/Republican_Party_(United_States)',
 'id': '35b31d9b4f723f806fd32662ef29edf7',
 'path': '35b31d9b4f723f806fd32662ef29edf7.jpg',
 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=227x227>}